# Terminations in Microsoft Autogen

## Why Termination Matters

We have used team to do some work, say write a story or accomplish some task. 

In [1]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [31]:
from autogen_agentchat.agents import AssistantAgent
add_1_agent_first = AssistantAgent(
    name = 'add_1_agent_first',
    model_client=model_client,
    system_message="Add 1 to the number, first number is 0. Give result as output"
)

add_1_agent_second = AssistantAgent(
    name = 'add_1_agent_second',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)
 
add_1_agent_third = AssistantAgent(
    name = 'add_1_agent_third',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)

from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third]
)

In [33]:
from autogen_agentchat.ui import Console

await Console(team.run_stream())

Error processing publish message for add_1_agent_first_0b0d3891-30b0-4f58-b2db-da0cc2a0674b/0b0d3891-30b0-4f58-b2db-da0cc2a0674b
Traceback (most recent call last):
  File "/Users/admin/Agentic-AI/Autogen/autogen-env/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 510, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/Agentic-AI/Autogen/autogen-env/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/Agentic-AI/Autogen/autogen-env/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/Agentic-AI/Autogen/autogen-env/lib/python3.12/site-packages/autoge

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [32]:
from autogen_agentchat.conditions import MaxMessageTermination

max_termination = MaxMessageTermination(5)

In [4]:
team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],
    termination_condition =max_termination
)

In [5]:
from autogen_agentchat.ui import Console

await Console(team.run_stream())

---------- add_1_agent_first ----------
1
---------- add_1_agent_second ----------
2
---------- add_1_agent_third ----------
3
---------- add_1_agent_first ----------
4
---------- add_1_agent_second ----------
5


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=2), metadata={}, content='1', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=2), metadata={}, content='2', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=39, completion_tokens=2), metadata={}, content='3', type='TextMessage'), TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=55, completion_tokens=2), metadata={}, content='4', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=60, completion_tokens=2), metadata={}, content='5', type='TextMessage')], stop_reason='Maximum number of messages 5 reached, current message count: 5')

In [4]:
from autogen_agentchat.agents import AssistantAgent

In [5]:

agent1 = AssistantAgent(
    name = 'story_writer',
    model_client=model_client,
    system_message="Give the story about a brave knight, keep it short no more than 40 words. if critic say 'THE END' anywhere. Only output 'THE END'"
)

agent2 = AssistantAgent(
    name = 'story_critic',
    model_client=model_client,
    system_message="Continue the story and critic it with feedback. Keep it short and no more than 40 words. If it feels complete, just say 'THE END'. Only output 'THE END'"
)

In [5]:
from autogen_agentchat.conditions import TextMentionTermination


text_mention_termination = TextMentionTermination('THE END')
teamWithTextTermination = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=text_mention_termination
)

In [6]:
from autogen_agentchat.ui import Console

await Console(teamWithTextTermination.run_stream(task = 'Write a story about a brave knight.'))

---------- user ----------
Write a story about a brave knight.


---------- story_writer ----------
Sir Cedric, the brave knight, faced the fire-breathing dragon threatening his kingdom. With unwavering courage, he charged, sword gleaming. Against overwhelming odds, he triumphed, saving the realm and earning eternal admiration for his valor and selflessness.
---------- story_critic ----------
THE END


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a story about a brave knight.', type='TextMessage'), TextMessage(source='story_writer', models_usage=RequestUsage(prompt_tokens=53, completion_tokens=54), metadata={}, content='Sir Cedric, the brave knight, faced the fire-breathing dragon threatening his kingdom. With unwavering courage, he charged, sword gleaming. Against overwhelming odds, he triumphed, saving the realm and earning eternal admiration for his valor and selflessness.', type='TextMessage'), TextMessage(source='story_critic', models_usage=RequestUsage(prompt_tokens=117, completion_tokens=3), metadata={}, content='THE END', type='TextMessage')], stop_reason="Text 'THE END' mentioned")

# Combining Termination Conditions

In [11]:
combined_termination = MaxMessageTermination(5) | TextMentionTermination('THE END')

from autogen_agentchat.conditions import TextMentionTermination


text_mention_termination = TextMentionTermination('THE END')
teamWithTextTermination = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=combined_termination
)

In [13]:
from autogen_agentchat.ui import Console

await Console(teamWithTextTermination.run_stream(task = 'Write a story about a brave knight.'))

---------- user ----------
Write a story about a brave knight.
---------- story_writer ----------
Sir Rowan, the fearless knight, embarked on a quest to rescue the captured princess. Battling treacherous terrain and cunning foes, his determination never faltered. Through relentless bravery, he succeeded, restoring hope and earning the kingdom's eternal gratitude.
---------- story_critic ----------
THE END


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a story about a brave knight.', type='TextMessage'), TextMessage(source='story_writer', models_usage=RequestUsage(prompt_tokens=302, completion_tokens=51), metadata={}, content="Sir Rowan, the fearless knight, embarked on a quest to rescue the captured princess. Battling treacherous terrain and cunning foes, his determination never faltered. Through relentless bravery, he succeeded, restoring hope and earning the kingdom's eternal gratitude.", type='TextMessage'), TextMessage(source='story_critic', models_usage=RequestUsage(prompt_tokens=383, completion_tokens=3), metadata={}, content='THE END', type='TextMessage')], stop_reason="Text 'THE END' mentioned")

# External Termination

ExternalTermination: Enables programmatic control of termination from outside the run. This is useful for UI integration (e.g., “Stop” buttons in chat interfaces).

In [6]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [ ]:
from autogen_agentchat.agents import AssistantAgent

agent1 = AssistantAgent(
    name = 'story_writer',
    model_client=model_client,
    system_message="Give the story about a brave knight, keep it short no more than 40 words. if critic say 'THE END' anywhere. Only output 'THE END'"
)

agent2 = AssistantAgent(
    name = 'story_critic',
    model_client=model_client,
    system_message="Continue the story and critic it with feedback. Keep it short and no more than 40 words. If it feels complete, just say 'THE END'. Only output 'THE END'"
)

from autogen_agentchat.conditions import ExternalTermination
external_termination = ExternalTermination()

from autogen_agentchat.teams import RoundRobinGroupChat
team = RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition= external_termination
)



In [17]:
from autogen_agentchat.ui import Console
run = asyncio.create_task(Console(team.run_stream(task = 'Write a story about a brave knight less than 40 words.')))

await asyncio.sleep(10)

external_termination.set()
await run

---------- user ----------
Write a story about a brave knight less than 40 words.
---------- story_critic ----------
Sir Alden, the brave knight, ventured into the enchanted forest. Facing countless perils, he retrieved the mythical gem, saving the kingdom from eternal darkness. His legacy shone brighter than the gem itself. THE END
---------- story_writer ----------
THE END
---------- story_critic ----------
THE END
---------- story_writer ----------
THE END
---------- story_critic ----------
THE END
---------- story_writer ----------
THE END
---------- story_critic ----------
THE END
---------- story_writer ----------
THE END
---------- story_critic ----------
THE END
---------- story_writer ----------
THE END
---------- story_critic ----------
THE END


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a story about a brave knight less than 40 words.', type='TextMessage'), TextMessage(source='story_critic', models_usage=RequestUsage(prompt_tokens=271, completion_tokens=45), metadata={}, content='Sir Alden, the brave knight, ventured into the enchanted forest. Facing countless perils, he retrieved the mythical gem, saving the kingdom from eternal darkness. His legacy shone brighter than the gem itself. THE END', type='TextMessage'), TextMessage(source='story_writer', models_usage=RequestUsage(prompt_tokens=319, completion_tokens=3), metadata={}, content='THE END', type='TextMessage'), TextMessage(source='story_critic', models_usage=RequestUsage(prompt_tokens=332, completion_tokens=3), metadata={}, content='THE END', type='TextMessage'), TextMessage(source='story_writer', models_usage=RequestUsage(prompt_tokens=338, completion_tokens=3), metadata={}, content='THE END', type='TextMessage'), Te

In [ ]:
# The team is not stopping immediately but rather current agent is completing its run.

# Aborting A Team

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [29]:
from autogen_core import CancellationToken

cancellation_token = CancellationToken()

run2 = asyncio.create_task(
    Console(team.run_stream(task = 'Give a short Story about a lion atmost 40 words',cancellation_token=cancellation_token))
)

await asyncio.sleep(5)
cancellation_token.cancel()

try:
    result = await run2
except :
    print("Task Was Cancelled")

---------- user ----------
Give a short Story about a lion atmost 40 words


Error processing publish message for story_critic_6ef0ad3e-c13a-4219-9e7f-21a8d50305f6/6ef0ad3e-c13a-4219-9e7f-21a8d50305f6
Traceback (most recent call last):
  File "/Users/admin/Agentic-AI/Autogen/autogen-env/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 510, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/Agentic-AI/Autogen/autogen-env/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 113, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/Agentic-AI/Autogen/autogen-env/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/admin/Agentic-AI/Autogen/autogen-env/lib/python3.12/site-packages/autogen_cor

Task Was Cancelled
